# Spark Connected Components

In [1]:
import os, sys, time

# SPARK_HOME="/home1/01131/tg804093/src/spark-1.4.1-bin-hadoop2.6"
SPARK_HOME="/home1/01131/tg804093/src/supercomputing2015-tutorial/02_hadoop_on_hpc/work/spark-1.4.1-bin-hadoop2.6"
os.environ["SPARK_HOME"]=SPARK_HOME
sys.path.insert(0, os.path.join(SPARK_HOME, "python"))
sys.path.insert(0, os.path.join(SPARK_HOME, 'python/lib/py4j-0.8.2.1-src.zip')) 
sys.path.insert(0, os.path.join(SPARK_HOME, 'bin') )

# import Spark Libraries
from pyspark import SparkContext, SparkConf, Accumulator, AccumulatorParam
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.mllib.linalg import Vector

try:
    sc
except NameError:
    sc = SparkContext("local[1]")
    sqlCtx=SQLContext(sc)

print "Loaded Spark: %s"%(sc.version)

Loaded Spark: 1.4.1


In [2]:
OUTPUT_DIR="/work/01131/tg804093/graph"
files = os.listdir(OUTPUT_DIR)
print str(files)
for i in files:
    filename = os.path.join(OUTPUT_DIR, i)

['graph_edges_10000_29735.csv', 'graph_edges_1000000_2983785.csv', 'graph_edges_10000000_29860787.csv', 'graph_edges_10000_29752.csv', 'graph_edges_100000_298441.csv']


In [3]:
%%time

start = time.time()
filename = os.path.join(OUTPUT_DIR, "graph_edges_10000000_29860787.csv")
data = sc.textFile(filename).map(lambda line: [int(line.split(",")[0]), int(line.split(",")[1])])
# add backward edges
data = data.flatMap(lambda v: [(v[0],v[1]),(v[1],v[0])])
data_grouped = data.groupByKey().mapValues(lambda a: sorted(set(a))).cache()
end = time.time()
                                               
read_time = end - start
print "Read Time: %.1f sec"%read_time 

Read Time: 0.6 sec
CPU times: user 11 ms, sys: 3 ms, total: 14 ms
Wall time: 615 ms


In [17]:
data_grouped.first()

(0, [15978, 22569, 39097, 73662, 78836])

In [4]:
%%time
new_iteration_needed = sc.accumulator(0)
# check for smaller keys in each set
def process_vertex(vertex):
    """ pass single vertex and its adjecent vertices
        e.g.: (0, [0, 67, 14])
    """
    global new_iteration_needed
    source = vertex[0]
    local_max = False
    
    first_edge_destination = vertex[1][0]
    new_vertices = []    
    print "*********Source: %d First Edge Dest: %d"%(source, first_edge_destination) 
    if source <= first_edge_destination:
        local_max = True
        new_vertices.append((source, first_edge_destination))
            
    print "Process: " + str(vertex) + " Local Max: " + str(local_max)
    last_edge_destination = first_edge_destination

    for current_destination in vertex[1]:
        #print "Current destination: %s"%str(current_destination)
        if current_destination == last_edge_destination: 
            continue
        
        if local_max == True:
            edge = (source, current_destination)
            new_vertices.append(edge)
        else:
            new_vertices.append((first_edge_destination, current_destination))
            new_vertices.append((current_destination, first_edge_destination))
            print "Add 1 to accumulator"
            new_iteration_needed.add(1)

        last_edge_destination = current_destination
    
    if ((not local_max) and (source < last_edge_destination)):
        new_vertices.append((source, first_edge_destination))
    
    #print "Return new vertices: " + str(new_vertices)
    return new_vertices


#process_vertex((19, [7, 9, 19, 41]))
num_iterations=0
cc = data_grouped
start = time.time()
while True:
    old_accum_value = new_iteration_needed.value
    print "*********** Start iteration: %d " % num_iterations
    #print "Accum before iteration: " + str(old_accum_value)
    cc = cc.flatMap(lambda v: process_vertex(v))\
           .groupByKey()\
           .mapValues(lambda a: sorted(set(a)))
    cc.collect()
    num_iterations = num_iterations + 1
    #print "New iteration accum: %d old value: %d"%(new_iteration_needed.value, old_accum_value)
    if old_accum_value < new_iteration_needed.value:
        #print "Accumulator value was increased. New iteration."
        continue
    else:
        break
end = time.time()

print "Finished after %d Iterations. Found %d components. Time: %.2f"%(num_iterations, cc.count(), (end-start))

*********** Start iteration: 0 


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 34, localhost): java.lang.OutOfMemoryError: Java heap space
	at java.util.Arrays.copyOf(Arrays.java:3236)
	at java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:113)
	at java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:93)
	at java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:140)
	at java.io.ObjectOutputStream$BlockDataOutputStream.write(ObjectOutputStream.java:1853)
	at java.io.ObjectOutputStream.write(ObjectOutputStream.java:709)
	at org.apache.spark.util.Utils$.writeByteBuffer(Utils.scala:187)
	at org.apache.spark.scheduler.DirectTaskResult$$anonfun$writeExternal$1.apply$mcV$sp(TaskResult.scala:51)
	at org.apache.spark.util.Utils$.tryOrIOException(Utils.scala:1239)
	at org.apach